# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233938185567                   -0.50    8.0         
  2   -7.250265360505       -1.79       -1.40    1.0   6.81ms
  3   -7.251147911001       -3.05       -2.15    1.0   7.19ms
  4   -7.251201539951       -4.27       -2.08    2.0   8.61ms
  5   -7.251332103056       -3.88       -2.66    1.0   7.08ms
  6   -7.251338075620       -5.22       -3.15    2.0   8.36ms
  7   -7.251338735641       -6.18       -3.73    1.0   7.19ms
  8   -7.251338787217       -7.29       -3.95    3.0   9.89ms
  9   -7.251338796451       -8.03       -4.39    1.0   7.53ms
 10   -7.251338798377       -8.72       -4.88    2.0   8.79ms
 11   -7.251338798659       -9.55       -5.12    3.0   10.1ms
 12   -7.251338798691      -10.49       -5.46    1.0   7.87ms
 13   -7.251338798702      -10.95       -5.85    2.0   9.04ms
 14   -7.251338798704      -11.78       -6.19    1.0   7.53ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.070773891249633
[ Info: Arnoldi iteration step 2: normres = 0.5253487917555449
[ Info: Arnoldi iteration step 3: normres = 0.7009193051488015
[ Info: Arnoldi iteration step 4: normres = 0.22435740486280956
[ Info: Arnoldi iteration step 5: normres = 0.5413094581547006
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.20e-02, 8.13e-02, 4.84e-01, 2.26e-01, 2.37e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2724298981869029
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.71e-03, 1.29e-01, 1.94e-01, 1.07e-01, 3.60e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0944096284759632
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.28e-04, 1.26e-02, 1.58e-02, 6.80e-02, 5.83e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10595890733453593
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.82e-06,